<a href="https://colab.research.google.com/github/kjh1311990/face/blob/main/Fuzzy_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
from google.colab import drive
drive.mount('/content/drive')

!pip install komm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
import komm
import numpy as np
code = komm.BCHCode(11,365)

In [141]:
import numpy as np

PARAMS_M = 2047
PARAMS_N = 512

PARAMS_LOGQ = 10
PARAMS_LOGL = 6

PARAMS_Q = 1 << PARAMS_LOGQ
PARAMS_L = 1 << PARAMS_LOGL

In [171]:
def EMBLEM_Encode(m, shift):
    encm = ((m << 1) + 1) << (shift - 1);
    return encm.reshape(PARAMS_M, 1);

def EMBLEM_Decode(m, shift):
    decm = m >> shift;
    return decm.reshape(PARAMS_M, 1);

In [194]:
from keras.models import load_model
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

def Setup():
  A = np.random.randint(low = -PARAMS_L/2, high = PARAMS_L/2+1, size = PARAMS_N * PARAMS_M, dtype = int)
  A.resize(PARAMS_M, PARAMS_N)

  return A;

def Encap(A, folder):
  model = load_model('/content/drive/MyDrive/Notebooks/weights.h5')
  model2 = Model(model.input, model.layers[-2].output)

  img = np.zeros((5,224,224,3));
  feature = np.zeros((5,512));

  for i in range(0,5):
    img[i] = image.load_img(folder + '/' + str(i) +'.jpg', target_size=(224, 224))
    x = image.img_to_array(img[i]);
    x = np.expand_dims(x, axis=0);
    x = preprocess_input(x);
    feature[i] = model2.predict(x);

  t = (feature.max(axis=0) - feature.min(axis=0));

  for i in range(0,5):
    for j in range(0,512):
      if t[j] > 1:
        feature[i][j] = (feature[i][j] / (4* t[j]))

  feature = feature.astype(int);

  b = feature[0].reshape(PARAMS_N,1);

  m = np.random.randint(low = 0, high = 2, size = 133, dtype = int);

  encm = code.encode(m);
  encm.reshape(2047,1);

  c = np.remainder(np.matmul(A,b) + EMBLEM_Encode(encm, PARAMS_LOGQ-1), PARAMS_Q);

  return (t, c, m)

def Decap(A, t, c, imgg):
  img = image.load_img(imgg, target_size=(224, 224))
  x = image.img_to_array(img);
  x = np.expand_dims(x, axis=0);
  x = preprocess_input(x);
  feature = model2.predict(x);
  feature = feature.reshape(PARAMS_N,1);

  for j in range(0,512):
    if t[j] > 1:
      feature[j] = (feature[j] / (4* t[j]))

  bprime = feature.astype(int);

  cprime = np.remainder(c - np.matmul(A, bprime), PARAMS_Q)
  decm = EMBLEM_Decode(cprime, PARAMS_LOGQ-1).reshape(2047)
  mprime = code.decode(decm);
  return mprime;

In [180]:
A = Setup();

In [ ]:
(t, c, m) = Encap(A, '/content/drive/MyDrive/Notebooks/face/output/1');

In [195]:
mprime = Decap(A, t, c, '/content/drive/MyDrive/Notebooks/face/output/1/1.jpg');

In [196]:
m

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1])

In [197]:
mprime

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1])

In [193]:
mprime2 = Decap(A, c, '/content/drive/MyDrive/Notebooks/face/output/2/1.jpg');

In [198]:
mprime2

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0])

In [200]:
np.sum(np.abs(m - mprime2))

62